In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models

from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from load_data import load_data

# from torch.utils.data.sampler import SubsetRandomSampler

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## AlexNet from scratch

In [2]:
# AlexNet model architecture initialisation

class AlexNet(nn.Module):
    def __init__(self, nbr_classes, Dropout):
        super(AlexNet, self).__init__()
        self.L1= nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11,stride=4),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size=3, stride = 2)
        )
        self.L2= nn.Sequential(
            nn.Conv2d(96,256,kernel_size= 5),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3,2)
        )
        self.L3= nn.Sequential(
            nn.Conv2d(256,384,3),
            nn.ReLU(),
            nn.BatchNorm2d(384)
        )
        self.L4= nn.Sequential(
            nn.Conv2d(384,384,3),
            nn.ReLU(),
            nn.BatchNorm2d(384)
        )
        self.L5= nn.Sequential(
            nn.Conv2d(384, 256,3),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3,2),
        )
        self.FC= nn.Sequential(
            nn.Flatten(),
            nn.Dropout(Dropout),
            nn.Linear(256,4096),
            nn.Dropout(Dropout),
            nn.Linear(4096,4096),
            nn.Dropout(Dropout),
            nn.Linear(4096,nbr_classes),
            nn.Softmax(nbr_classes)   # 15 persons for now just to test apres nzido     
        )
        
    def forward(self, data):
        out = self.L1(data)
        out = self.L2(out)
        out = self.L3(out)
        out = self.L4(out)
        out = self.L5(out)
        out = self.FC(out)
        return out

In [3]:
# Hyperparametres
NC = 108
epochs = 10
batch_size = 30
learning_rate = 0.01
Dr = 0.2

# Optimizer, loss
model = AlexNet(nbr_classes= NC, Dropout= Dr).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.005, betas=[0.9,0.9])   # betas hna houwa momentum 9oli
loss = nn.CrossEntropyLoss()


In [4]:
# Create and Load data
trans = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

data = load_data(csv_file='casia.csv', transformer= trans)

traind, testd = random_split(data, [700, 55])

train_loader = DataLoader(traind, batch_size , shuffle= True)
test_loader = DataLoader(testd, batch_size , shuffle= True)

In [5]:
data.__getitem__(15)

(tensor([[[0.1686, 0.1255, 0.1765,  ..., 0.1922, 0.2000, 0.2275],
          [0.1412, 0.1608, 0.1882,  ..., 0.2275, 0.2235, 0.2157],
          [0.1647, 0.1647, 0.1490,  ..., 0.2275, 0.2431, 0.2275],
          ...,
          [0.2941, 0.3333, 0.3451,  ..., 0.1490, 0.1686, 0.2157],
          [0.2902, 0.3373, 0.3922,  ..., 0.1294, 0.1294, 0.1569],
          [0.3333, 0.3490, 0.3686,  ..., 0.1451, 0.1373, 0.1529]]]),
 tensor(2))

In [6]:
# Train the model
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader): 
        
        images=images.to(device)
        labels=labels.to(device)
        #  img= img.unsqueeze(0)    # turns an n.d. tensor into an (n+1).d. one by adding an extra dimension of depth 1
        
        pred = model(images)
        err= loss(pred, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        err.backward()
        optimizer.step()
        
        print(f'Epoch [{epoch}/{epochs}], Loss : {err.item()}')

        

Epoch [0/10], Loss : 4.749927997589111
Epoch [0/10], Loss : 222.2673797607422
Epoch [0/10], Loss : 332.9625549316406
Epoch [0/10], Loss : 971.3455200195312
Epoch [0/10], Loss : 1665.962646484375
Epoch [0/10], Loss : 563.7496948242188


OSError: Unsupported BMP pixel depth (264)

In [ ]:
with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy : {} %'.format(14, 100 * correct / total)) 

Accuracy of the network on the 14 validation images: 0.0 %


In [ ]:
# Pretrained model in pytorch
model = models.alexnet()

